In [1]:
###Python form recognizer analyze layout###

import json
import time
from requests import get, post

#Endpoint urlo

endpoint = r"https://form-recog-demo-ms.cognitiveservices.azure.com/"
apim_key = "13a22a02075145e5ac378bdfe7725892"
post_url = endpoint + "/formrecognizer/v2.1/Layout/analyze"
source = r"/Users/mukesh_kumar_singh/Documents/image_to_text/input_files/5201564372.tif"

headers = {
    'Content-Type' : 'image/tif',
    'Ocp-Apim-Subscription-Key': apim_key,
}

with open(source, "rb") as f:
    data_bytes = f.read()
    
try:
    resp = post(url=post_url, data = data_bytes, headers=headers)
    if resp.status_code !=202:
        print("POST analyze failed:\n%s" % resp.text)
        quit()
    print("POST analyze succeded:/n%s" % resp.headers)
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

POST analyze succeded:/n{'Content-Length': '0', 'Operation-Location': 'https://form-recog-demo-ms.cognitiveservices.azure.com/formrecognizer/v2.1/layout/analyzeResults/c433b826-522f-47d2-83c8-22bb73770ad9', 'x-envoy-upstream-service-time': '344', 'apim-request-id': 'c433b826-522f-47d2-83c8-22bb73770ad9', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'East US', 'Date': 'Fri, 10 Feb 2023 16:13:18 GMT'}


In [2]:
n_tries = 10
n_try = 0
wait_sec = 10
resp_json = ""
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = {'Ocp-Apim-Subscription-Key': apim_key})
        resp_json = json.loads(resp.text)
        print("Running")
        if resp.status_code !=200:
            print("GET Layout results failed:\n%s" % resp_json)
            break
        status = resp_json["status"]
        if status == "succeeded":
            print("Succeeded")
            
            file = open(r"/Users/mukesh_kumar_singh/Documents/image_to_text/json_convert.json", "w")
            file.write(json.dumps(resp.json()))
            file.close()
            break
        if status == "failed":
            print("Layout Analysis failed:\n%s" % resp_json)
            break
        time.sleep(wait_sec)
        n_try += 1
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        break

Running
Running
Running
Running
Running
Running
Running
Succeeded


In [7]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.options.display.max_columns = None

#for read_result in resp_json["analyzeResult"]["readResults"]:
#    print("Page No: %s" % read_result['page'])
#    print("-------Page %d: Extracted OCR ---------" % read_result["page"])
#    for line in read_result["lines"]:
#        print(line["text"])

dataframe_list = []
for pageresult in resp_json["analyzeResult"]["pageResults"]:
    for table in pageresult['tables']:
        if pageresult["page"] > 0 and table["cells"][0]["text"] == 'PURCHASE ORDER':
            print("-----Page %d: Extracted table------" % pageresult["page"])
            print("No of Rows: %s" % table["rows"])
            print("No of Columns: %s" % table["columns"])
            if table["rows"]>2:
                tableList = [[None for x in range(table["columns"])] for y in range(table["rows"])]
                for cell in table['cells']:
                    tableList[cell["rowIndex"]][cell["columnIndex"]]=cell["text"]
                df = pd.DataFrame.from_records(tableList)
                df = df.drop(df[df[3].isnull() 
                                        & df[4].isnull() 
                                        & df[5].isnull()].index).reset_index(drop=True)
                # Below code gives list of columns having more than 60% null
                df = df.replace([0,'', None, 'NULL'],np.nan)
                #null_percentage = df.isnull().sum()/df.shape[0]*100
                #col_to_drop = null_percentage[null_percentage>75].keys()
                #print(col_to_drop)
                #col_to_drop = col_to_drop.delete(0)
                #col_to_drop = col_to_drop.delete(0)
                #output_df = df.drop(col_to_drop, axis=1)
                ### Below code is to drop rows and clean up row level data
                #output_df.columns = ["PURCHASE ORDER", "LINE #", "Trx Worker Name & No", "Description", "Work Date", "Bill Type", "Quantity"]
                #output_df = output_df[1:]
                #output_df = output_df.drop(output_df[output_df['Description'].isnull() 
                #                        & output_df['Work Date'].isnull() 
                #                        & output_df['Bill Type'].isnull()].index).reset_index(drop=True)
                #dataframe_list.append(output_df)
                #data = pd.concat([data, output_df])
                display(df)
                #data = data.append(output_df)
                
#print(dataframe_list)
#final_data = pd.concat(dataframe_list)
##Applying forward fill to the dataframes
#final_data = final_data.fillna(method='ffill')
#display(final_data)
##output data to a storage
#final_data.to_csv('/Users/mukesh_kumar_singh/Documents/image_to_text/out_files/5201564372.csv', index=False)  



-----Page 2: Extracted table------
No of Rows: 5
No of Columns: 5


KeyError: 5

In [31]:
import pandas as pd
from IPython.display import display

pd.options.display.max_columns = None

#for read_result in resp_json["analyzeResult"]["readResults"]:
#    print("Page No: %s" % read_result['page'])
#    print("-------Page %d: Extracted OCR ---------" % read_result["page"])
#    for line in read_result["lines"]:
#        print(line["text"])
        
for pageresult in resp_json["analyzeResult"]["pageResults"]:
    if pageresult["page"] > 3 :
        for table in pageresult['tables']:
            print("-----Page %d: Extracted table------" % pageresult["page"])
            print("No of Rows: %s" % table["rows"])
            print("No of Columns: %s" % table["columns"])
            if table["rows"]>0:
                tableList = [[None for x in range(table["columns"])] for y in range(table["rows"])]
                for cell in table['cells']:
                    tableList[cell["rowIndex"]][cell["columnIndex"]]=cell["text"]
                df = pd.DataFrame.from_records(tableList)
                display(df)

-----Page 4: Extracted table------
No of Rows: 48
No of Columns: 9


,0,1,2,3,4,5,6,7,8
0,PURCHASE ORDER,LINE #,Trx Worker Name & No,None,Description,Work Date,Bill Type,None,Quantity
1,4501097370,10,347507 Stewart Cornelius,None,Leadhand - Compulsory,"Nov 7, 2022",Regular,None,8.50
2,,,"Nov 7, 2022",None,,,,None,8.50
3,,,347507 Stewart Cornelius,None,Leadhand - Compulsory,"Nov 8, 2022",Regular,None,10.00
4,,,"Nov 8, 2022",,,,,,10.00
5,,,347507 Stewart Cornelius,None,Leadhand - Compulsory,"Nov 9, 2022",Regular,None,10.00
6,,,"Nov 9, 2022",,,,,None,10.00
7,,,347507 Stewart Cornelius,,Leadhand - Compulsory,"Nov 10, 2022",,Regular,10.00
8,,,"Nov 10, 2022",,,,,None,10.00
9,,,347507 Stewart Cornelius,,Leadhand - Compulsory,"Nov 15, 2022",Regular,None,10.00


-----Page 5: Extracted table------
No of Rows: 48
No of Columns: 7


,0,1,2,3,4,5,6
0,PURCHASE ORDER,LINE #,Trx Worker Name & No,Description,Work Date,Bill Type,Quantity
1,,,347703 Gerald Jennex,Planner,"Nov 8, 2022",Regular,10.00
2,,,"Nov 8, 2022",,,,10.00
3,,,347703 Gerald Jennex,Planner,"Nov 9, 2022",Regular,10.00
4,,,"Nov 9, 2022",,,,10.00
5,,,347703 Gerald Jennex,Planner,"Nov 10, 2022",Regular,10.00
6,,,"Nov 10, 2022",,,,10.00
7,,,347703 Gerald Jennex,Planner,"Nov 15, 2022",Regular,10.00
8,,,"Nov 15, 2022",,,,10.00
9,,,347703 Gerald Jennex,Planner,"Nov 16, 2022",Regular,10.00


-----Page 6: Extracted table------
No of Rows: 48
No of Columns: 8


...

,0,1,2,3,4,5,6
0,Row Labels,Night Shift,Subsistence,Employee Initials,Total Hours,Regular Hours,OT Hours
1,"COUGHLIN, DARRELL",,,,10.0,10.0,
2,"DEAN, JEFF",,,,10.0,10.0,
3,"HOUSER, SCOTT",,,,10.0,10.0,
4,"JENNEX, GERALD W.",,,,9.5,9.5,
5,"KUNTZ, CURTIS",,,,10.0,10.0,
6,"SMITH, RYAN E",,,,10.0,10.0,
7,"WILLIAMS, ROBERT",,,,10.0,10.0,
8,Grand Total,,,,69.5,69.5,


-----Page 22: Extracted table------
No of Rows: 3
No of Columns: 7


,0,1,2,3,4,5,6
0,Row Labels,Night Shift,Subsistence,Employee Initials,Total Hours,Regular Hours,OT Hours
1,"Cornelius, Stewart",,,RW,10.0,10.0,
2,Grand Total,,,,10.0,10.0,


-----Page 23: Extracted table------
No of Rows: 3
No of Columns: 7


,0,1,2,3,4,5,6
0,Row Labels,Night Shift,Subsistence,Employee Initials,Total Hours,Regular Hours,OT Hours
1,"LOVE, TAYLOR",,,,10.0,10.0,
2,Grand Total,,,,10.0,10.0,
